# Prompt Engineering LLM with Learning by Examples

Adapter le modele génératif de langage pour ajouter un module "auto-critic". Concretement, cela consiste à ajouter une sortie à la derniere sequence de decodeur. Entrainer le modele en figeant les poids du LLM (TransfertLearning), puis fineTuner le modele complet.

![LLM-Critic](LLM-AutoCritic.png)

**Dans cette deuxième étape, nous explorerons l'interaction entre le modèle de langage pré-entraîné (LLM) et une source externe à travers un exemple d'application. Nous aborderons également la manière de structurer les "prompts" de manière optimale pour atteindre notre objectif. Il est important de noter que ce document doit être exécuté dans Colab afin de bénéficier des capacités de calcul nécessaires pour mener à bien les tâches.**


*Nous aborderons les points suivants : :*

    - Comment utiliser un logiciel de chaine de prompt ?
    - Comment lancer un agent et une conversation avec un LLM ?
    
 ### Bibliographie :
 
 
 - [Reasoning & Thought](https://arxiv.org/abs/2212.09597) (review)
 - [LangChain](https://python.langchain.com/docs/get_started/introduction.html) et [ReAct](https://arxiv.org/abs/2210.03629)
 - [Tutoriel](https://larevueia.fr/langchain-le-guide-essentiel/) et [vidéo](https://youtu.be/mAoNANPOsd0)
 
 
 ### Prérequis : 

    - Python 3.8
    - LangChain
    - 8Gb RAM

In [2]:
import torch

**Pour Colab :**

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!nvidia-smi

## Import Model :

Nous utilisons un modèle "instruct" ajusté pour la compréhension des instructions et est spécialisé dans l'interaction humaine, notamment dans les discussions (Chat). Le modèle *Falcon-7B-instruct* est efficace pour les techniques de prompts d'instruction, pour plus de détail, voir la partie conclusion et perspective. Les calculs ont été effectués sur Google Colab en utilisant une carte graphique NVIDIA Tesla T4.

LangChain est un framework qui simplifie la création d'applications exploitant de vastes modèles de langage. Une des fonctionnalités clés de LangChain est sa capacité à interagir avec les pipelines de modèles HuggingFace, facilitant ainsi l'utilisation de ces modèles dans le processus.

In [1]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline

Pour implémenter le modele Falcon-7B-Instruct, nous faisons :

In [3]:
model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Pour préparer un modèle de pipeline afin de l'utiliser avec LangChain, nous faisons :

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

Le paramètre de température ajuste le caractère aléatoire de la sortie, à 0, le modèle n'a pas d'aléatoire.

## Automatize Prompt :

LangChain offre une fonctionnalité essentielle : l'automatisation de la construction de chaînes de prompts. Les étapes de la chaîne correspondent au raisonnement du modèle de langage. Avec LangChain, il est possible de définir un prompt "fixe" contenant des variables d'entrée, que l'on peut indiquer sous la forme d'une chaîne de caractères formatée. Cela permet une flexibilité dans l'interaction avec le modèle et facilite l'adaptation du prompt en fonction des besoins spécifiques de chaque demande.

In [5]:
from langchain import PromptTemplate, LLMChain

In [6]:
template = """
You are an intelligent chatbot. Help the following question with brilliant answers.
Question: {question}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["question"])

Pour importer un modèle de langage pré-entraîné avec un prompt spécifique, vous pouvez suivre l'étape suivante :

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

Et enfin, pour générer le résultat :

In [ ]:
question = "Explain what is Artificial Intellience as Nursery Rhymes "
print(llm_chain.run(question))

C'est grâce à la composabilité de l'outil et des prompts que vous pouvez construire votre propre cas d'utilisation. Cette fonctionnalité vous permet de combiner différentes étapes et prompts pour créer des workflows personnalisés répondant à vos besoins spécifiques.

### Agent : Math Solver example

Un agent est un composant qui a accès à une suite d'outils et peut décider quel outil utiliser en fonction de l'entrée de l'utilisateur. Il existe deux principaux types d'agents : les "agents d'action" et les "agents de planification et d'exécution". Les agents d'action décident d'une action à entreprendre et exécutent cette action une étape à la fois.

Cet outil nous permet, par exemple, de construire un agent capable d'interagir avec Internet pour effectuer des recherches et créer automatiquement une documentation (comme dans l'exemple de l'Agent-GPT), ainsi que d'implémenter une fonctionnalité pour exécuter du code (comme dans l'exemple de l'Auto-GPT). Dans notre cas, nous allons simplement créer un agent qui résout des problèmes de mathématiques. Pour importer et paramétrer des agents pour les mathématiques, vous pouvez suivre les étapes suivantes :

In [ ]:
from langchain.agents import load_tools

In [ ]:
tools = load_tools(
    ['llm-math'],
    llm=llm
)

On initialiser l'agent de cette façon :

In [ ]:
from langchain.agents import initialize_agent

In [ ]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

L'agent utilisé, appelé "zero-shot-react-description", est basé sur le framework ReAct (Reasoning + Acting), présenté dans un article de recherche de 2022. Pour tester cet agent, vous pouvez suivre les étapes suivantes :

In [ ]:
zero_shot_agent("Combient fait (4.5*2.1)^2.2 ?")

Veuillez noter que cette approche n'est pas adaptée aux problèmes non mathématiques, car elle ne sera pas en mesure de trouver des valeurs numériques ou des mesures de similarité d'embedding*. Cependant, il existe d'autres agents adaptés à cette tâche. Pour ajouter un agent spécifique, vous pouvez suivre les étapes suivantes :

In [ ]:
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

In [ ]:
tools.append(llm_tool)

Avec cet agent, il est possible d'engager des conversations, mais il n'est pas spécifiquement conçu pour automatiser la mémorisation. Pour cette fonctionnalité, il existe un module plus avancé qui répond mieux à ce besoin.

*Les embeddings sont des représentations numériques des relations entre les chaînes de texte, exprimées sous forme de vecteurs de nombres réels. La distance entre deux vecteurs est utilisée pour mesurer leur proximité ; plus la distance est petite, plus la similarité est élevée. 

### Memory : Conversation

Pour réaliser des conversations, nous utilisons directement le module complet dédié suivant :

In [ ]:
from langchain.chains import ConversationChain

L'initialisation se fait de la manière suivante :

In [ ]:
conversation = ConversationChain(llm=llm)
print(conversation.prompt.template)

On observe deux paramètres importants : {history} et {input}. Le paramètre {history} contient les informations de la conversation précédente. Il existe trois types de mémoires disponibles :

    - ConversationBufferMemory : cette option conserve l'intégralité de la conversation précédente, mais peut épuiser rapidement le nombre de jetons disponibles.
    - ConversationSummaryMemory : cette option résume la conversation en quelques lignes, mais la qualité de synthèse dépend des capacités du modèle de langage.
    - ConversationBufferWindowMemory : cette option sélectionne plusieurs séquences aléatoires de la conversation précédente pour en conserver une fenêtre de mémoire.

Ces différents types de mémoires permettent de choisir la meilleure approche en fonction des besoins spécifiques de la conversation.

## Conclusion et Perspective

Nous avons abordé la génération de prompts et la construction d'agent LLM. En combinant ces concepts avec le fine-tuning, il est possible de construire un modèle d'instruction/chat créant des données, notamment avec l'exemple de l'auto-apprentissage (Voir [self-instruct](https://arxiv.org/abs/2109.02846)). En outre, LangChain pourrait simplifier également l'utilisation d'algorithmes tels que les "[Tree of Thought](https://github.com/princeton-nlp/tree-of-thought-llm)" (arbre de pensées) ou les "[Forests of Thought](https://github.com/mrspiggot/forestOfThoughts)" (forêts de pensées), qui offrent des approches structurées pour la génération de texte et la résolution de problèmes.

Les modèles de langage LLM présentent encore certaines limitations dans leur capacité à généraliser les problèmes (Voir cet [article](https://arxiv.org/pdf/2302.14045.pdf)). C'est pourquoi l'approche des MLLM (MultiModal Large Language Model, voir [1](https://github.com/HenryHZY/Awesome-Multimodal-LLM) et [2](https://github.com/BradyFU/Awesome-Multimodal-Large-Language-Models)) devient pertinente en combinant des adaptateurs et un LLM en tant que base (backbone). Cette approche offre la possibilité d'ajouter des fonctionnalités supplémentaires telles que l'apprentissage par renforcement, la curiosité ou la modélisation de la causalité, ce qui pourrait considérablement améliorer la généralisation des modèles (en particulier dans le cadre des *General Game Playing*). 